In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [2]:
def str_from_date(date_time):
    """
    datetime object to 2016_04_28 string
    """
    return '_'.join(str(date_time.date()).split('-'))

In [3]:
date = datetime.datetime(2015, 2, 28)
next_date = datetime.datetime(2015, 3, 28)

In [4]:
PRODUCTS_COUNT = 24
HEADER = ["ncodpers", "ind_ahor_fin_ult1", "ind_aval_fin_ult1",
          "ind_cco_fin_ult1", "ind_cder_fin_ult1", "ind_cno_fin_ult1",
          "ind_ctju_fin_ult1", "ind_ctma_fin_ult1", "ind_ctop_fin_ult1",
          "ind_ctpp_fin_ult1", "ind_deco_fin_ult1", "ind_deme_fin_ult1",
          "ind_dela_fin_ult1", "ind_ecue_fin_ult1", "ind_fond_fin_ult1",
          "ind_hip_fin_ult1", "ind_plan_fin_ult1", "ind_pres_fin_ult1",
          "ind_reca_fin_ult1", "ind_tjcr_fin_ult1", "ind_valo_fin_ult1",
          "ind_viv_fin_ult1", "ind_nomina_ult1", "ind_nom_pens_ult1",
          "ind_recibo_ult1"]

In [16]:
present = pd.read_csv('data/train_' + str_from_date(date) + '.csv', header=None, names=HEADER, usecols=[1]+range(24,48), index_col=0)

In [17]:
future = pd.read_csv('data/train_'+str_from_date(next_date) + '.csv', header=None, names=HEADER, usecols=[1]+range(24,48), index_col=0)

In [18]:
train = pd.DataFrame()

In [22]:
present.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
15890,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,1,1
15892,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,0,1
15893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
15894,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,1,1,1


In [5]:
def get_added_products(a, b):
    # a = list(present.loc[n,:])
    # b = list(present.loc[n,:])
    return [x[0] for x in filter(lambda x: x[1] == 1, zip(HEADER[1:], [(x^y)&y for x,y in zip(a, b)]))]

In [6]:
def create_row(n, present, future, train):
    try:
        a = present.loc[n,:]
        b = future.loc[n,:]
    except KeyError:
        return train
    added_products = get_added_products(list(a), list(b))
    for added_product in added_products:
        a['added_product'] = added_product
        train = train.append(a)
    return train

In [14]:
train = pd.DataFrame()

In [102]:
for i in tqdm(present.index):
    train = create_row(i, train)

100%|██████████| 925076/925076 [15:22<00:00, 1002.43it/s]


In [7]:
def get_month_next_month(year, month):
    if month == 12:
        next_m = 1
        next_y = year + 1
    else:
        next_m = month + 1
        next_y = year
    
    return datetime.datetime(year, month, 28), datetime.datetime(next_y, next_m, 28)

In [8]:
def get_added_products_factor(year, month):
    print year, month
    present_month, future_month = get_month_next_month(year, month)
    present = pd.read_csv('data/train_' + str_from_date(present_month) + '.csv', header=None, names=HEADER, usecols=[1]+range(24,48), index_col=0)
    
    future = pd.read_csv('data/train_' + str_from_date(future_month) + '.csv', header=None, names=HEADER, usecols=[1]+range(24,48), index_col=0)
    present.fillna(0, inplace=True)
    future.fillna(0, inplace=True)
    for col in HEADER[1:]:
        present[col] = present[col].astype(int)
        future[col] = future[col].astype(int)
    train = pd.DataFrame()
    for i in tqdm(present.index):
        train = create_row(i, present, future, train)
    train.to_csv('data/added_product_' + str_from_date(present_month) + '.csv', columns=['added_product'], index_label='ncodpers')
    return "Success"

In [9]:
months = [(2015, x) for x in range(11, 13)] + [(2016, x) for x in range(1, 5)]

In [112]:
train.to_csv('data/test_added_product.csv', columns=['added_product'], index_label='ncodpers')

In [10]:
map(lambda x: get_added_products_factor(x[0], x[1]), months[:])

2015 11


100%|██████████| 906109/906109 [14:34<00:00, 90.64it/s]


2015 12


100%|██████████| 912021/912021 [11:33<00:00, 99.16it/s]


2016 1


...

2016 3


100%|██████████| 925076/925076 [12:01<00:00, 1282.32it/s]


2016 4


100%|██████████| 928274/928274 [12:39<00:00, 1222.17it/s]


['Success', 'Success', 'Success', 'Success', 'Success', 'Success']

In [1]:
a = [0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0, 0, 0,1]
b = [0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0, 0, 0,0]
c = [0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0, 0, 0,0]

In [10]:
get_added_products_factor(2015, 2)

2015 2


100%|██████████| 627394/627394 [10:48<00:00, 967.12it/s]


'Success'